In [88]:
from scipy.io import loadmat
import numpy as np

# Load the .mat file
data = loadmat('data_temp.mat')

# Explore the structure of the file
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'c_p1', 'c_p3', 'co_gr_p1', 'co_gr_p3', 'corre', 'group_p1', 'group_p2', 'group_p3', 'ncorre', 'ngroup_p1', 'ngroup_p2', 'ngroup_p3', 'r_p1', 'r_p2', 'r_p3'])


In [89]:
data['__header__']

b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Jun 15 14:25:36 2017'

In [90]:
data['__version__']

'1.0'

In [91]:
data['__globals__']

[]

In [92]:
# Access and print each dataset
c_p1 = data['c_p1']
c_p3 = data['c_p3']
co_gr_p1 = data['co_gr_p1']
co_gr_p3 = data['co_gr_p3']
corre = data['corre']
group_p1 = data['group_p1']
group_p2 = data['group_p2']
group_p3 = data['group_p3']
ncorre = data['ncorre']
ngroup_p1 = data['ngroup_p1']
ngroup_p2 = data['ngroup_p2']
ngroup_p3 = data['ngroup_p3']
r_p1 = data['r_p1']
r_p2 = data['r_p2']
r_p3 = data['r_p3']

# Example: Print the shape of each dataset
print("c_p1 shape:", c_p1.shape)
print("c_p3 shape:", c_p3.shape)
print("co_gr_p1 shape:", co_gr_p1.shape)
print("co_gr_p3 shape:", co_gr_p3.shape)
print("corre shape:", corre.shape)
print("group_p1 shape:", group_p1.shape)
print("group_p2 shape:", group_p2.shape)
print("group_p3 shape:", group_p3.shape)
print("ncorre shape:", ncorre.shape)
print("ngroup_p1 shape:", ngroup_p1.shape)
print("ngroup_p2 shape:", ngroup_p2.shape)
print("ngroup_p3 shape:", ngroup_p3.shape)
print("r_p1 shape:", r_p1.shape)
print("r_p2 shape:", r_p2.shape)
print("r_p3 shape:", r_p3.shape)

c_p1 shape: (5180, 8)
c_p3 shape: (5180, 8)
co_gr_p1 shape: (5, 4, 280)
co_gr_p3 shape: (5, 4, 280)
corre shape: (1, 8)
group_p1 shape: (5, 4, 280)
group_p2 shape: (4, 280)
group_p3 shape: (5, 4, 280)
ncorre shape: (1, 8)
ngroup_p1 shape: (5, 4, 280)
ngroup_p2 shape: (4, 280)
ngroup_p3 shape: (5, 4, 280)
r_p1 shape: (5180, 8)
r_p2 shape: (946, 4)
r_p3 shape: (5180, 8)


### Chequeo de non-parametric normalization

In [93]:
group_p1[0, 0, 0]

153.0

In [94]:
ngroup_p1[0, 0, 0]

1.8918918918918919

Entonces usando los datos de la `Supplementary Table 1`:
- $n_i = \frac{x_i - median(x)}{median(|x-median(X)|)}$
- $x_i=153$
- $median(x)=81$
- $median(|x-median(X)|)=38$

$\therefore \frac{153 - 81}{38} \approx 1.8947$

### Data de interés
 
- **`group_p1`(5, 4, 280)**: respuestas en el stage i1 (individual answers). `group_p1[i, p, g]` es la respuesta del integrante `i` a la pregunta `p` perteneciente al grupo `g`.
  
- **`group_p2`(4, 280)**: respuestas en el stage c (collective answers). `group_p2[p, g]` es la respuesta a la pregunta `p` en el grupo `g`.

- **`group_p3`(5, 4, 280)**: respuestas en el stage i2 (individual revised answers). `group_p3[i, p, g]` es la respuesta revisada del integrante `i` a la pregunta `p` perteneciente al  grupo `g`.

- **`co_gr_p1`(5, 4, 280)**: nivel de confianza (del 1 al 10) de los 1400 participanes en el stage i1 (individual answers). `co_gr_p1[i, p, g]` es el nivel de confianza del integrante `i` para la respuesta a la pregunta `p` en el grupo `g`.
  
- **`co_gr_p3`(5, 4, 280)**: nivel de confianza (del 1 al 10) de los 1400 participanes en el stage i3 (individual revised answers). `co_gr_p3[i, p, g]` es el nivel de confianza del integrante `i` para la respuesta a la pregunta `p` en el grupo `g`.

- **`ngroup_p1`(5, 4, 280)**: datos normalizados usando el método `Non-parametric normalization` explicado en la sección *Methods* del paper Navajas (2018).
  
- **`ngroup_p2`(4, 280)**: datos normalizados usando el método `Non-parametric normalization` explicado en la sección *Methods* del paper Navajas (2018).

- **`ngroup_p3`(5, 4, 280)**: datos normalizados usando el método `Non-parametric normalization` explicado en la sección *Methods* del paper Navajas (2018).

### Chequeo datos NO normalizados

In [114]:
transposed_arr = group_p1.transpose(1, 0, 2) # para que qude (4, 5, 280)
group_p1_flattened = transposed_arr.reshape(4, -1) # para que quede (4, 1400)

In [115]:
# respuesta del individuo 0 en original
print(group_p1[0, 0, 0])
print(group_p1[0, 1, 0])
print(group_p1[0, 2, 0])
print(group_p1[0, 3, 0])

153.0
15.0
340.0
100000.0


In [116]:
# respuesta del individuo 0 en reshaped
group_p1_flattened[:, 0]

array([1.53e+02, 1.50e+01, 3.40e+02, 1.00e+05])

In [117]:
group_p1_flattened = group_p1_flattened.T
nan_mask = np.isnan(group_p1_flattened)

# NaNs x columna
nan_count_per_column = np.sum(nan_mask, axis=0)
print("nans por columna en group_p1")
print(nan_count_per_column)

nans por columna en group_p1
[12 21 41 68]


### Chequeo datos normalizados

In [107]:
transposed_arr = ngroup_p1.transpose(1, 0, 2) # para que qude (4, 5, 280)
ngroup_p1_flattened = transposed_arr.reshape(4, -1) # para que quede (4, 1400)

In [108]:
# respuesta del individuo 0 en original
print(ngroup_p1[0, 0, 0])
print(ngroup_p1[0, 1, 0])
print(ngroup_p1[0, 2, 0])
print(ngroup_p1[0, 3, 0])

1.8918918918918919
-0.6666666666666666
-0.24
nan


In [109]:
# respuesta del individuo 0 en reshaped
ngroup_p1_flattened[:, 0]

array([ 1.89189189, -0.66666667, -0.24      ,         nan])

In [110]:
ngroup_p1_flattened = ngroup_p1_flattened.T
nan_mask = np.isnan(ngroup_p1_flattened)

# NaNs x columna
nan_count_per_column = np.sum(nan_mask, axis=0)
print("nans por columna en ngroup_p1")
print(nan_count_per_column)

nans por columna en ngroup_p1
[ 12  21  64 272]
